Найти топ 5% пользователей с наибольшим количеством заказов по name

In [1]:
import pandas as pd
import duckdb

In [2]:
orders = pd.DataFrame({
    "order_id": list(range(1, 41)),
    "user_id": [
    5,5,5,5,5,5,5,5,5,5,      # 10 заказов (Ева)
    2,2,2,2,2,2,2,2,           # 8 заказов (Саша)
    1,1,1,1,1,1,               # 6 заказов (Алиса)
    3,3,3,3,                   # 4 заказа (Лена)
    4,4,4,                      # 3 заказа (Давид)
    6,7,8,9,10,11,12,13,14      # по одному заказу
],
    "state": ["delivered"] * 40   
})

In [3]:
users = pd.DataFrame({
    "user_id": list(range(1, 21)),
    "name": [
        "Алиса", "Саша", "Лена", "Давид", "Ева",
        "Марина", "Игорь", "Оля", "Иван", "Юля",
        "София", "Саша", "Олег", "Лера", "Миша",
        "Наташа", "Ирина", "Петр", "Лиза", "Рита"
    ] 
})

In [6]:
orders.head()

,order_id,user_id,state
0,1,5,delivered
1,2,5,delivered
2,3,5,delivered
3,4,5,delivered
4,5,5,delivered


In [7]:
users.head()

,user_id,name
0,1,Алиса
1,2,Саша
2,3,Лена
3,4,Давид
4,5,Ева


In [17]:
query = """
with user_orders as (
  select u.user_id,
         u.name,
         count(o.order_id) as cnt
  from users u
  left join orders o on u.user_id = o.user_id
  group by u.user_id, u.name
),
ranked as (
  select uo.*,
         row_number() over (order by uo.cnt desc) as rn,
         (select count(*) from user_orders) as total_users
  from user_orders uo
)
select user_id, name, cnt
from ranked
where rn <= ceil(total_users * 0.05)
"""

result = duckdb.query(query).to_df()
result

,user_id,name,cnt
0,5,Ева,10


 Есть еще вариант считать что под "топ-5%" имеют в виду тех, у кого количество заказов входит в верхние 5% по значению, а не по позиции в списке. Тогда нужно считать чей cnt ≥ 95-й перцентиль. 

In [4]:
query = """
with user_orders as ( 
  select u.user_id, u.name, count(o.order_id) as cnt
  from users u
  left join orders o on u.user_id = o.user_id
  group by u.user_id, u.name
),
threshold as (
  select approx_quantile(cnt, 0.95) as p95 
  from user_orders
)
select user_id, name, cnt
from user_orders uo
cross join threshold t
where uo.cnt > t.p95
"""
result = duckdb.query(query).to_df()
result

,user_id,name,cnt
0,5,Ева,10


In [6]:
query = """
with user_orders as ( 
  select u.user_id,
         u.name,
         count(o.order_id) as cnt
  from users u
  left join orders o on u.user_id = o.user_id
  group by u.user_id, u.name
),
ranked as (
  select uo.*,
         cume_dist() over (order by cnt) as cd
  from user_orders uo
)
select user_id, name, cnt
from ranked
where cd > 0.95;
"""
result = duckdb.query(query).to_df()
result

,user_id,name,cnt
0,5,Ева,10
